# 1. Synthetic to real domain gap
Take some real image crops from  Sam， with less occlusions，as the templates. And mach them with the raw crops from Sam. You can maybe apply a threshold for the Bounding box size to filter the valid template. See if you can get some reasonable results

## 1.1 Take and save some real image crops from SAM
Take input image then put via sam then choose some of the crops- save in the cnos analysis folder

In [1]:
# load sam
from src.model.sam import CustomSamAutomaticMaskGenerator, load_sam
from segment_anything.modeling.sam import Sam
model_type = "vit_h"
checkpoint_dir =  "datasets/bop23_challenge/pretrained/segment-anything"
sam_model = load_sam(model_type, checkpoint_dir)
custom_sam_model = CustomSamAutomaticMaskGenerator(sam=sam_model)

In [3]:
def move_to_device(segmentor_model, device="cuda:0"):
    # if there is predictor in the model, move it to device
    if hasattr(segmentor_model, "predictor"):
        segmentor_model.predictor.model = (
            segmentor_model.predictor.model.to(device)
        )
    else:
        segmentor_model.model.setup_model(device=device, verbose=True)
move_to_device(custom_sam_model)

In [22]:
from PIL import Image
import torch
import numpy as np
from src.model.utils import Detections
rgb_path = "datasets/bop23_challenge/datasets/icbin/test/000001/rgb/000001.png"
rgb = Image.open(rgb_path).convert("RGB")
detections = custom_sam_model.generate_masks(np.array(rgb))

In [12]:
def extract_object_by_mask(image, mask, width: int = 512):
    mask = Image.fromarray(mask)
    masked_image = Image.composite(
        image, Image.new("RGB", image.size, (0, 0, 0)), mask)
    cropped_image = masked_image.crop(masked_image.getbbox())
    # new_height = width * cropped_image.height // cropped_image.width
    return cropped_image

masked_images = []
for mask in detections["masks"].cpu():
    binary_mask = np.array(mask) * 255
    binary_mask = binary_mask.astype(np.uint8)
    masked_image = extract_object_by_mask(image, binary_mask)
    masked_images.append(masked_image)

In [17]:
# Visualize proposals
import matplotlib.pyplot as plt
from math import ceil
def plot_images(images, rows, cols):
    fig, axes = plt.subplots(rows, cols, figsize=(20, 30))
    for i, ax in enumerate(axes.flat):
        if i >= len(images):
            break
        ax.imshow(images[i])
        ax.axis('off')
    plt.tight_layout()
    plt.show()

# Define the number of rows and columns in the grid
rows = ceil(len(masked_images) / 6)
cols = 6
# Plot the masked_images array in a grid
# plot_images(masked_images, rows, cols) 

In [30]:
# We are choosing proposals 2 and 8
crop1 = {
    "crop" : masked_images[1], # it is saved as Image so crop1["crop"] to visualize it
    "masks" : detections["masks"][1],
    "boxes" : detections["boxes"][1]
}
crop2 = {
    "crop" : masked_images[7],
    "masks" : detections["masks"][7],
    "boxes" : detections["boxes"][7]
}

# save image
crop1["crop"].save("cnos_analysis/crop_proposals/crop1.png")
crop2["crop"].save("cnos_analysis/crop_proposals/crop2.png")

## 1.2 Create and save 42 templates from the test folder using BlenderProc.

In [ ]:
# load config file
import hydra
from omegaconf import DictConfig, OmegaConf
from hydra import initialize, compose

# Initialize Hydra and compose the configuration
initialize(config_path="configs")
cfg = compose(config_name="run_inference")

In [43]:
# 10* set_struct to so that we can add key-value to the file - see lb we add new key-value root_dir to ref_dataloader_config
OmegaConf.set_struct(cfg, False)

In [44]:
default_ref_dataloader_config = cfg.data.reference_dataloader
ref_dataloader_config = default_ref_dataloader_config.copy()
ref_dataloader_config._target_ = "src.dataloader.bop_pbr.BOPTemplatePBR"
ref_dataloader_config.root_dir = "datasets/bop23_challenge/datasets"
ref_dataloader_config.template_dir += "templates_pyrender/icbin" # we r working on icbin dataset
ref_dataloader_config.templates_output_folder = "cnos_analysis/real_images_templates"


In [46]:
from hydra.utils import instantiate
ref_dataset = instantiate(ref_dataloader_config) # src.dataloader.bop.BOPTemplatePBR
ref_dataset.load_processed_metaData(reset_metaData=True)

INFO:lightning_lite.utilities.seed:Global seed set to 2023
INFO:lightning_lite.utilities.seed:Global seed set to 2023


InstantiationException: Error in call to target 'src.dataloader.bop_pbr.BOPTemplatePBR':
FileNotFoundError(2, 'No such file or directory')
full_key: data.reference_dataloader

# 2. Scale
Try to zoom in and zoom out the templates or real image crops for the Sam instance， see if the similarly score changes

# 3. Occlusion
Find a real image crop without occlusion， and maskout some area of it，see how much the similarly score decrease